In [3]:
from pycaret.regression import *

In [4]:
import pandas as pd
import numpy as np
import pandas_datareader as pdr
from datetime import datetime

In [29]:
COINS=['ADA-USD']

data = pdr.yahoo.daily.YahooDailyReader(symbols=COINS, start=datetime(2008, 11, 9), end=datetime(2022, 8, 27), get_actions=True)
df = data.read()
df=df.iloc[:, df.columns.get_level_values(0)=='Close']

In [30]:
df

Attributes,Close
Symbols,ADA-USD
Date,
2017-11-09,0.032053
2017-11-10,0.027119
2017-11-11,0.027437
2017-11-12,0.023977
2017-11-13,0.025808
...,...
2022-08-24,0.458109
2022-08-25,0.464999


In [31]:
df = df.droplevel(level=1,axis=1)

In [32]:
df['Series'] = np.arange(1,len(df)+1)

In [33]:
df = df.reset_index()

In [34]:
df

Attributes,Date,Close,Series
0,2017-11-09,0.032053,1
1,2017-11-10,0.027119,2
2,2017-11-11,0.027437,3
3,2017-11-12,0.023977,4
4,2017-11-13,0.025808,5
...,...,...,...
1749,2022-08-24,0.458109,1750
1750,2022-08-25,0.464999,1751
1751,2022-08-26,0.430863,1752
1752,2022-08-27,0.449703,1753


In [35]:
df['Month'] = [i.month for i in df['Date']]

In [36]:
df['Year'] = [i.year for i in df['Date']]

In [37]:
df

Attributes,Date,Close,Series,Month,Year
0,2017-11-09,0.032053,1,11,2017
1,2017-11-10,0.027119,2,11,2017
2,2017-11-11,0.027437,3,11,2017
3,2017-11-12,0.023977,4,11,2017
4,2017-11-13,0.025808,5,11,2017
...,...,...,...,...,...
1749,2022-08-24,0.458109,1750,8,2022
1750,2022-08-25,0.464999,1751,8,2022
1751,2022-08-26,0.430863,1752,8,2022
1752,2022-08-27,0.449703,1753,8,2022


In [38]:
df.drop(['Date'], axis=1, inplace=True)

In [39]:
df= df[['Series', 'Year', 'Month', 'Close']] 

In [40]:
df.head(5)

Attributes,Series,Year,Month,Close
0,1,2017,11,0.032053
1,2,2017,11,0.027119
2,3,2017,11,0.027437
3,4,2017,11,0.023977
4,5,2017,11,0.025808


In [41]:
train = df[df['Series'] <= (len(df)*0.90) ]
test = df[df['Series'] > (len(df)*0.90)]

In [42]:
train.shape, test.shape

((1578, 4), (176, 4))

In [43]:
s = setup(data = train,
          test_data = test,
          target = "Close", 
          fold_strategy = 'timeseries', 
          numeric_features = ['Year', 'Series'], 
          fold = 3, 
          transform_target = True, 
          session_id = 123)

,Description,Value
0,session_id,123
1,Target,Close
2,Original Data,"(1578, 4)"
3,Missing Values,False
4,Numeric Features,2
5,Categorical Features,1
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(1578, 14)"


In [44]:
df['Close'] 

0       0.032053
1       0.027119
2       0.027437
3       0.023977
4       0.025808
          ...   
1749    0.458109
1750    0.464999
1751    0.430863
1752    0.449703
1753    0.430049
Name: Close, Length: 1754, dtype: float64

In [45]:
best = compare_models(sort = 'MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,0.4097,0.5165,0.4594,-2.4462,0.2416,0.5360,0.0233
rf,Random Forest Regressor,0.4109,0.5223,0.4611,-2.4061,0.2427,0.5238,0.1167
lightgbm,Light Gradient Boosting Machine,0.4160,0.5346,0.4650,-2.2471,0.2455,0.5028,0.0367
et,Extra Trees Regressor,0.4183,0.5413,0.4678,-2.3853,0.2478,0.5431,0.0900
gbr,Gradient Boosting Regressor,0.4242,0.5539,0.4736,-2.5465,0.2523,0.5341,0.0400
dt,Decision Tree Regressor,0.4262,0.5609,0.4765,-2.5969,0.2542,0.5414,0.0133
ada,AdaBoost Regressor,0.4418,0.5982,0.4889,-2.5332,0.2638,0.5208,0.0500
par,Passive Aggressive Regressor,0.5122,0.7831,0.5540,-3.4622,0.3199,0.6442,0.0167
lasso,Lasso Regression,0.5301,0.8241,0.5709,-3.7139,0.3363,0.6751,1.2033
en,Elastic Net,0.5302,0.8242,0.5710,-3.7277,0.3364,0.6762,1.1633


In [46]:
prediction_holdout = predict_model(best);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,K Neighbors Regressor,0.2934,0.1085,0.3293,-1.0846,0.1931,0.5494


In [47]:
predictions = predict_model(best, data=df)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,K Neighbors Regressor,0.0423,0.0118,0.1088,0.9715,0.0625,0.0810


In [48]:
predictions

Attributes,Series,Year,Month,Close,Label
0,1,2017,11,0.032053,0.027116
1,2,2017,11,0.027119,0.027116
2,3,2017,11,0.027437,0.027116
3,4,2017,11,0.023977,0.026076
4,5,2017,11,0.025808,0.025947
...,...,...,...,...,...
1749,1750,2022,8,0.458109,0.901040
1750,1751,2022,8,0.464999,0.901040
1751,1752,2022,8,0.430863,0.901040
1752,1753,2022,8,0.449703,0.901040


In [49]:
import plotly.express as px

In [50]:
fig = px.line(predictions, x='Series', y=["Close", "Label"], template = 'plotly_dark')

In [51]:
fig.show()

In [52]:
test

Attributes,Series,Year,Month,Close
1578,1579,2022,3,0.823752
1579,1580,2022,3,0.797379
1580,1581,2022,3,0.801250
1581,1582,2022,3,0.849293
1582,1583,2022,3,0.806316
...,...,...,...,...
1749,1750,2022,8,0.458109
1750,1751,2022,8,0.464999
1751,1752,2022,8,0.430863
1752,1753,2022,8,0.449703
